<a href="https://colab.research.google.com/github/jvnolasco/Imersao_Alura_IA/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
%pip -q install google-genai

In [34]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [33]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [32]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
  model = MODEL_ID,
  contents='Qual é a próxima imersão de IA com Google Gemini da Alura?'

)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura está agendada para começar no dia **27 de maio de 2024**.


In [31]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição já se encerraram no dia 11 de maio. Fique de olho no site da Alura para futuras edições!


In [30]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini data']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br, hipsters.network



In [29]:
# Instalar Framework ADK de agentes do Google ################################################

!pip install -q google-adk


In [28]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [27]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [26]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################


def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramente de busca do google (google_search) para recuperar as ultimas noticias de lançamento muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possivel que ele não seja tão relevante assim e pode ser substituido por outro que tenha mais.
      Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos_buscados



In [24]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscador, você deve:
        usar ferramentas de busca do Google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles, Você também
        pode usar o (google_search) para encotrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais releante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [43]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [44]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [45]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Por favor, insira um tópico válido.")
else:
  print(f"Maravilha,! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- 📈​ Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------")


  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📈​ Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------")


  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- 📈​ Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- 📈​ Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))




🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: futebol
Maravilha,! Vamos então criar o post sobre novidades em futebol

--- 📈​ Resultado do Agente 1 (Buscador) ---



> Ok, aqui estão as notícias sobre os 5 lançamentos mais relevantes no mundo do futebol no último mês:
> 
>  
> Com certeza! Aqui estão 5 lançamentos de notícias relevantes do mundo do futebol no último mês:
> 
>  1. **Transferências de jogadores:** Tem havido muita atividade no mercado de transferências, com vários jogadores de alto nível se mudando para novos clubes. Por exemplo, o Real Madrid está perto de contratar o defesa Dean Huijsen por cerca de 60 milhões de euros, enquanto o Friburgo contratou o avançado Cyriaque Irié. Além disso, o SC Braga renovou o contrato de João Moutinho por mais uma temporada.
>  2. **Novos Estádios:** Vários clubes estão construindo ou renovando seus estádios. O Manchester United anunciou planos para construir um novo estádio no lugar do Old Trafford, enquanto o Estrela da Amadora planeja construir um novo estádio com capacidade para 12.000 espectadores. O Real Madrid também está renovando o Estádio Santiago Bernabéu, e o Villarreal CF está atualizando o Estadio de la Cerámica.
>  3. **Competições e Resultados:** Várias competições de futebol estão em andamento em todo o mundo. Na Copa Libertadores, São Paulo e Botafogo tiveram resultados positivos. Na Liga 3 Placard, o Sporting B garantiu a promoção para a Liga Portugal 2 Meu Super. Além disso, a Copa KTO de Futebol Master está programada para começar em 24 de maio.
>  4. **Jovens Jogadores:** Há vários jovens jogadores surgindo no mundo do futebol. Gabriel Silva (Sporting CP), Lucas Taibo (Sporting), Gustavo Varela (Benfica) são jovens promissores.
>  5. **Disputa Bola de Ouro:** Depois de uma das disputas mais acirradas dos últimos anos, Rodri levou a melhor na Bola de Ouro em 2024. No entanto, devido a uma lesão, a disputa pelo prêmio ficou aberta para uma série de jogadores que querem suceder o espanhol.
> 


--------------------------------------------------------

--- 📈​ Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos de notícias fornecidos, vou elaborar um plano de conteúdo para cada um deles, identificar o tema mais relevante e, em seguida, propor um plano detalhado para um post sobre esse tema.
> 
> **Plano de Conteúdo para cada lançamento:**
> 
> 1.  **Transferências de Jogadores:**
>     *   Pontos Relevantes: Movimentações de grandes jogadores, valores das transferências, impacto nos clubes, renovações de contrato de jogadores experientes.
>     *   Possíveis Tópicos para Post:
>         *   Análise das maiores transferências e o que esperar dos jogadores em seus novos clubes.
>         *   Impacto financeiro das transferências nos clubes envolvidos.
>         *   Renovações de contrato e a importância da experiência no futebol.
> 
> 2.  **Novos Estádios:**
>     *   Pontos Relevantes: Construção e renovação de estádios, impacto na experiência dos torcedores, modernização das instalações, benefícios econômicos para as cidades.
>     *   Possíveis Tópicos para Post:
>         *   Detalhes sobre os novos estádios e suas tecnologias.
>         *   Impacto da modernização dos estádios na experiência dos torcedores.
>         *   Benefícios econômicos da construção e renovação de estádios para as cidades.
> 
> 3.  **Competições e Resultados:**
>     *   Pontos Relevantes: Resultados de competições importantes, classificações, destaques de jogadores, surpresas e zebras.
>     *   Possíveis Tópicos para Post:
>         *   Resumo das principais competições e seus resultados mais recentes.
>         *   Análise do desempenho dos times e jogadores em destaque.
>         *   Previsões e expectativas para as próximas rodadas.
> 
> 4.  **Jovens Jogadores:**
>     *   Pontos Relevantes: Descoberta de novos talentos, desempenho em categorias de base, potencial de crescimento, impacto nos clubes.
>     *   Possíveis Tópicos para Post:
>         *   Apresentação dos jovens talentos e suas habilidades.
>         *   Análise do potencial de crescimento e impacto nos clubes.
>         *   Comparação com jogadores já estabelecidos e projeções futuras.
> 
> 5.  **Disputa Bola de Ouro:**
>     *   Pontos Relevantes: Candidatos ao prêmio, desempenho ao longo da temporada, lesões e seus impactos, favoritos e azarões.
>     *   Possíveis Tópicos para Post:
>         *   Análise dos principais candidatos à Bola de Ouro.
>         *   Desempenho dos jogadores ao longo da temporada e seus momentos decisivos.
>         *   Impacto das lesões na disputa pelo prêmio.
> 
> **Tema Mais Relevante:**
> 
> Considerando o apelo global e o interesse contínuo dos fãs, a **Disputa da Bola de Ouro** parece ser o tema mais relevante. Este tópico gera muita discussão e engajamento, pois envolve a avaliação dos melhores jogadores do mundo e suas performances.
> 
> **Plano Detalhado para Post sobre a Disputa da Bola de Ouro:**
> 
> *   **Título:** A Nova Corrida pela Bola de Ouro: Quem Sucederá Rodri em 2025?
> *   **Introdução:**
>     *   Contextualizar a disputa da Bola de Ouro em 2025.
>     *   Mencionar a lesão de Rodri e como isso abre a disputa.
>     *   Apresentar os principais candidatos ao prêmio.
> *   **Tópicos a serem abordados:**
>     1.  **Análise dos Principais Candidatos:**
>         *   Nomear e detalhar os principais jogadores na disputa.
>         *   Estatísticas e desempenho na temporada até o momento.
>         *   Momentos decisivos e jogos de destaque.
>     2.  **Fatores que Influenciam a Escolha:**
>         *   Desempenho em ligas nacionais e competições europeias.
>         *   Títulos conquistados e impacto nos resultados do time.
>         *   Lesões e ausências importantes.
>     3.  **Azarões e Surpresas:**
>         *   Apresentar jogadores que podem surpreender na disputa.
>         *   Analisar o potencial de crescimento e impacto desses jogadores.
>         *   Destacar momentos em que esses jogadores se destacaram.
>     4.  **O Legado de Rodri:**
>         *   Relembrar a conquista de Rodri em 2024.
>         *   Analisar o impacto de sua lesão na disputa atual.
>         *   Comparar seu desempenho com os candidatos atuais.
>     5.  **Enquetes e Interação com o Público:**
>         *   Criar enquetes para que os seguidores votem em seus favoritos.
>         *   Incentivar comentários e discussões sobre os candidatos.
> *   **Conclusão:**
>     *   Resumir os principais pontos da disputa.
>     *   Incentivar a discussão e o debate entre os seguidores.
>     *   Fazer uma projeção de quem pode levar o prêmio e por quê.
> 
> Este plano oferece uma estrutura abrangente para um post que irá atrair a atenção dos fãs de futebol e gerar engajamento nas redes sociais.
> 


--------------------------------------------------------

--- 📈​ Resultado do Agente 3 (Redator) ---



> ✨ A NOVA CORRIDA PELA BOLA DE OURO COMEÇOU! ✨
> 
> Com a temporada a todo vapor, a pergunta que não sai da cabeça dos amantes do futebol é: quem vai brilhar e conquistar a Bola de Ouro em 2025? 🏆
> 
> A lesão do último detentor, Rodri, abriu uma lacuna e a disputa está mais emocionante do que nunca! 👀
> 
> 🔥 Nomes como Mbappé e Haaland já estão mostrando a que vieram, mas será que teremos alguma surpresa? Quem tem se destacado no seu time do coração?
> 
> 🤔 Quais critérios você acha que serão decisivos para a escolha do melhor do mundo? Desempenho individual, títulos ou momentos decisivos?
> 
> 💬 Deixe seu comentário com o seu palpite e vamos juntos nessa jornada rumo à Bola de Ouro!
> 
> \#BolaDeOuro \#Futebol \#MelhorDoMundo \#AluraEsporte
> 


--------------------------------------------------------

--- 📈​ Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!
